**Analytics goal:  Identifying the most frequent mbtags and their popularity**

In [1]:
%%local
import time
start = time.time()

In [2]:
import pyspark
import os
import re

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
62,application_1638946947785_0064,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
%%local
import plotly.graph_objects as go

In [4]:
%%local
import os
import re

from IPython.display import display
from ipywidgets import widgets, interactive
import ipywidgets as widgets
%config InlineBackend.figure_format = 'retina'
import time
import plotly.express as px
import warnings
warnings.filterwarnings("ignore")


In [5]:
rdd = sc.textFile("s3://millionsongs-data/new_g4g.csv")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
first_row = rdd.first()
input_rdd = rdd.filter(lambda x:x != first_row).map(lambda x : x.split('\t'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
input_rdd.cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

PythonRDD[3] at RDD at PythonRDD.scala:53

# mb_tags with highest song popularity

In [8]:
rdd_map=input_rdd.filter(lambda x : x[43]!= '')\
        .map(lambda x : (float(x[43]), re.findall(r"'(.*?)'", x[9], re.DOTALL) ))\
        .flatMapValues(lambda x :x)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
top_categories = rdd_map.map(lambda x : (x[1],x[0])).mapValues(lambda v: (v, 1))\
        .reduceByKey(lambda a,b : (a[0] +b[0], a[1] +b[1]),numPartitions=7)\
        .mapValues(lambda x : x[0]/x[1])\
        .sortBy(lambda x: x[1], ascending=False,numPartitions=7)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
top_categories_df = top_categories.toDF()
top_categories_df.createOrReplaceTempView('top_categories_view')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
%%sql -o df_popularity
SELECT * FROM top_categories_view order by _2 desc LIMIT 10

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [12]:
%%local
import plotly.express as px
import plotly.graph_objects as go
fig = go.Figure()
#data_canada = px.data.gapminder()
fig = px.bar(df_popularity[::-1], y='_1', x='_2', orientation='h', labels={
     "_1": "mb_tags",
     "_2": "Artist Popularity"},title="Mb_tags and Song Popularity",)
fig.show()

# WorldCloud of Top mbids with respect to their frequency

In [13]:
import re
mb_tags = input_rdd.map(lambda x: x[9])\
                .map(lambda x : re.findall(r"'(.*?)'", x, re.DOTALL) )\
                .flatMap(lambda x: x).map(lambda x : (x,1))\
                .reduceByKey(lambda x, y: x+y,numPartitions=7)\
                .sortBy(lambda x : x[1], ascending=False,numPartitions=7)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
mb_tags_dataframe = mb_tags.toDF()
mb_tags_dataframe.createOrReplaceTempView('mb_tags_view')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
%%sql -o mb_tags_df
SELECT * FROM mb_tags_view order by _2 DESC LIMIT 50

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Output()

In [16]:
%%local
import numpy as np
np.random.seed(5)
x = 10000*np.random.uniform(size = 50)
y = 10000*np.random.uniform(size = 50)
fig = px.scatter(mb_tags_df, x=x, 
                 y=y,
                 size =  [0]*len(mb_tags_df),
                 color = '_1',
                 #text = 'tags',
                 

                 color_discrete_sequence=px.colors.qualitative.Bold,
                  log_x=False, size_max=30)


for i in range(len(mb_tags_df)):
    annotation = { 
        # x -> location for x
        'x': x[i],
        # y -> location for y
        'y': y[i],
        'text': mb_tags_df['_1'][i],  # text
        'showarrow': False,  # would you want to see arrow
        'arrowhead': 3,  # which type for arrowhead
        'font': {'size': mb_tags_df['_2'][i]/50, 'color':'#9467bd'}  # font style
    }

    fig.add_annotation(annotation)
    

fig.update_layout(
    autosize=False,
    width=1000,
    height=800,
    margin=dict(
        l=50,
        r=50,
        b=30,
        t=40,
        pad=4
        
    ),
    plot_bgcolor = "white",
    paper_bgcolor="white"
)


fig.update_layout(showlegend=False)

fig.update_xaxes(visible=False)
  
fig.update_yaxes(visible=False)

In [17]:
%%local
stop = time.time()

In [18]:
%%local
print(f"time taken: {stop-start}")

time taken: 64.70321083068848


In [19]:
sc.stop()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…